In [4]:
import os, pandas, numpy, moment, xmlmanip, csv
os.environ['INTERFACE_CONF_FILE'] = '/home/jjorissen/interface_secrets.conf'
from tangier_api import api
from tangier_api.api import ScheduleConnection, ProviderConnection, ProviderReport, LocationConnection

In [5]:
site_id_file = '/home/jjorissen/Documents/TangierDocs/LocationSiteIds.xlsx'
site_ids = sorted(pandas.read_excel(site_id_file).dropna()['Site ID'].unique())
schedule = ScheduleConnection()

In [6]:
today = moment.utcnow().strftime("%Y-%m-%d")
three_months_ago = moment.utcnow().add(months=-2).strftime("%Y-%m-%d")
schedule_list = schedule.get_schedule_values_list(start_date=three_months_ago, 
                                                  end_date=today, 
                                                  site_ids=site_ids)

df = pandas.DataFrame(schedule_list)

In [7]:
schedule_df = df
provider_ids = list(schedule_df['empid'].unique())
provider_ids.remove(numpy.nan)
provider = ProviderConnection()
provider_df = pandas.DataFrame(provider.provider_info_values_list(provider_ids))
provider_df_columns = list(provider_df.columns)
provider_df_columns.remove('@action')
provider_df = provider_df[provider_df_columns]

In [8]:
location = LocationConnection()
location_df = pandas.DataFrame(location.location_info_values_list(site_ids=site_ids))
location_df_columns = list(location_df.columns)
location_df_columns.remove('@action')
location_df = location_df[location_df_columns]

In [9]:
schedule_df.to_csv('schedule_flat.csv', quoting=csv.QUOTE_ALL, index=False)
provider_df.to_csv('provider_flat.csv', quoting=csv.QUOTE_ALL, index=False)
location_df.to_csv('location_flat.csv', quoting=csv.QUOTE_ALL, index=False)